In [1]:
import numpy as np
import pandas as pd
import regex as re
import plotly as px
import plotly.graph_objects as go

In [2]:
wifi_responses = pd.read_csv('UCSD Wifi Poll (Responses) - Form Responses 1.csv')

In [3]:
## remove non ucsd emails
ucsd_emails_regex = '$@ucsd.edu'
wifi_ucsd = wifi_responses[wifi_responses['Email Address'].str.contains('@ucsd.edu', regex= True, na=False)]

## remove timestamp and email address
wifi_ucsd = wifi_ucsd.drop('Timestamp', axis=1).drop('Email Address', axis=1)
wifi_ucsd.head()

,College,How long have you had to wait for your WiFi to connect on campus during this quarter?,Where have you experienced the worst quality WiFi on campus this quarter?,Which WiFi server(s) have you experienced problems with? (Select all that apply),Has low quality WiFi impacted your studies or your class time?,"If so, how?"
0,ERC,10+ minutes,Center Hall,"UCSD-Protected, UCSD-Guest",Yes,Can't take notes in class and have to use my p...
1,Marshall,10+ minutes,dorms-marshall lowers,"UCSD-Protected, UCSD-Guest, Resnet (login requ...",Yes,NaN
2,Sixth,10+ minutes,York Hall,"UCSD-Protected, Resnet (login required), Resne...",Yes,In my TDPW 1 class we have had to wait up to 1...
3,Revelle,10+ minutes,"Price center, revelle plaza, basically in any ...","UCSD-Protected, UCSD-Guest, Resnet (login requ...",Yes,I have many one hour blocks between classes w...
4,Muir,No wait,ERC,"Resnet (login required), Resnet (no login requ...",No,NaN


In [4]:
## helper function to add eduroam in
def add_eduroam(wifi_servers):
    searched = re.search(r'[Ee]du', wifi_servers)
    if searched != None:
        removed = wifi_servers[:searched.start()]
        return removed + "Eduroam"
    elif (wifi_servers == "there is no wifi"):
        return "None"
    elif (wifi_servers == "Village East secure"):
        return "Resnet (login required)"
    return wifi_servers

## relevant locations
center = r'[Cc][Ee][Nn][Tt]'
peterson = r'[Pp][Ee][Tt][Ee][Rr]'
solis = r'[Ss][Oo][Ll][Ii][Ss]'
wlh = r'[Ww][Ll][Hh]'
wlh_full = r'[Ww][Aa][Rr]{2}[Ee][Nn] [Ll]ec'
york = r'[Yy]ork'
pcynh = r'[Pp][Cc][Yy]'
pcynh_full = r'[Pp]epper'
galbraith = r'[Gg][a][i]{0,1}lbr'
cse = r'[Cc][Ss][Ee]'
ebu3b = r'[Ee][Bb][Uu]'
ledden = r'[Ll][Ee][Dd]{2}'
mandeville = r'[Mm][Aa][Nn][Dd][Ee]'
argo = r'[Aa]rgo'
tioga = r'[Tt]ioga'
black = r'[Bb][Ll][Aa][Cc][Kk]'
muir = r'[Mm][Uu][Ii][Rr]'
warren = r'[Ww][Aa][Rr]{2}[Ee][Nn]'
revelle = r'[Rr][Ee][Vv][Ee][Ll]{2}'
erc = r'[Ee][Rr][Cc]'
sixth = r'[Ss][Ii][Xx][Tt][Hh]'
marshall_uppers = r'[Uu]pper'
marshall_lowers = r'[Ll]ower'
village = r'[Vv]illage'
sixty_four = r'64'
sixty_four_full = r'[Ss][Ii][Xx][Tt][Yy]'
pc = r'[Pp][Cc]'
pc_full = r'[Pp]rice'
geisel = r'[Gg][ei]{2}sel'
biomed = r'[Bb]iomed'

locations = {center: "CENTER", peterson: "PETER", solis: "SOLIS", wlh: "WLH", wlh_full: "WLH", york: "YORK",
             pcynh: "PCYNH", pcynh_full: "PCYNH", galbraith: "GH", cse: "CSE", ebu3b: "CSE", ledden: "LEDDN AUD",
             mandeville: "MDVL", argo: "ARGO", tioga: "TIOGA", black: "BLACK", muir: "MUIR", warren: "WARREN",
             revelle: "REVELLE", erc: "REVELLE", sixth: "SIXTH", marshall_uppers: "UPPERS", marshall_lowers: "LOWERS",
             village: "VILLAGE", sixty_four: "SIXTY FOUR", sixty_four_full: "SIXTY FOUR", pc: "PC", pc_full: "PC",
             geisel: "GEISEL", biomed: "BIOMED"}

## helper functions to filter locations
def filter_locations(wifi_locations):
    result = ''
    for key in locations:
        check = re.search(key, wifi_locations)
        if check != None:
            if result == '':
                result = locations[key]
            elif key == biomed:
                result += locations[key]
            else:
                result += ', ' + locations[key]
    return result

In [5]:
## map helper functions to their columns
wifi_ucsd['Where have you experienced the worst quality WiFi on campus this quarter?'] = wifi_ucsd[
    'Where have you experienced the worst quality WiFi on campus this quarter?'].apply(filter_locations)
wifi_ucsd['Which WiFi server(s) have you experienced problems with? (Select all that apply)'] = wifi_ucsd[
    'Which WiFi server(s) have you experienced problems with? (Select all that apply)'].apply(add_eduroam)

In [6]:
## connection times instances
connection_times = ['No wait','1-3 minutes','4-6 minutes','7-9 minutes', '10+ minutes']
total = 0
no_wait = 0
one_three = 0
four_six = 0
seven_nine = 0
more = 0

for time in wifi_ucsd['How long have you had to wait for your WiFi to connect on campus during this quarter?']:
    total += 1
    if time == 'No wait':
        no_wait += 1
    elif time == '1-3 minutes':
        one_three += 1
    elif time == '4-6 minutes':
        four_six += 1
    elif time == '7-9 minutes':
        seven_nine += 1
    else:
        more += 1

connection_times_val = [no_wait, one_three, four_six, seven_nine, more]
connection_times_val

[4, 39, 65, 43, 146]

In [ ]:
## visualizing connection times
colors = ['darkorange', 'lightcoral', 'lightgreen', 'gold', 'mediumturquoise']

fig = go.Figure(data=[go.Pie(labels=connection_times, values=connection_times_val)])

fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))

fig.show()

In [ ]:
## how much has wifi impacted studies negatively
impacted = 0
not_impacted = 0

for i in wifi_ucsd['Has low quality WiFi impacted your studies or your class time?']:
    if i == 'Yes':
        impacted += 1
    else:
        not_impacted += 1

wifi_impact = [impacted, not_impacted]
wifi_impact

In [ ]:
## wifi impact in percentages
total = sum(wifi_impact)
percentages = [impacted/total * 100, not_impacted/total * 100]
percentages

In [ ]:
## which wifi servers are the worst
wifi_servers = ['UCSD-Protected', 'UCSD-Guest', 'UCSD-Device', 'Resnet (login required)',
                'Resnet (no login required)', 'Eduroam']
protected = 0
guest = 0
device = 0
resnet_login = 0
resnet_nologin = 0
eduroam = 0

for value in wifi_ucsd['Which WiFi server(s) have you experienced problems with? (Select all that apply)']:
    if re.search(r'UCSD-Protected', value) != None:
        protected += 1
    if re.search(r'UCSD-Guest', value) != None:
        guest += 1
    if re.search(r'UCSD-Device', value) != None:
        device += 1
    if re.search(r'Resnet\s[(]login\srequired[)]', value) != None:
        resnet_login += 1
    if re.search(r'Resnet\s[(]no\slogin\srequired[)]', value) != None:
        resnet_nologin += 1
    if re.search(r'Eduroam', value) != None:
        eduroam += 1

worst_wifi_servers = [protected, guest, device, resnet_login, resnet_nologin, eduroam]
worst_wifi_servers

In [ ]:
total = sum(worst_wifi_servers)
percentages = [protected/total * 100, guest/total * 100, device/total * 100, resnet_login/total * 100, 
               resnet_nologin/total * 100, eduroam/total * 100]
percentages

In [ ]:
columns = {"Location": wifi_servers, "Count": worst_wifi_servers}
locations = pd.DataFrame(columns)
locations.to_csv('wifi_servers_data.csv', header=False, index=False)

In [ ]:
## find worst quality wifi on campus
places = {"CENTER": 0, "PETER": 0, "SOLIS": 0, "WLH": 0, "YORK": 0, "PCYNH": 0, "GH": 0, "CSE": 0, "LEDDN AUD": 0,
          "MDVL": 0, "ARGO": 0, "TIOGA": 0, "BLACK": 0, "MUIR": 0, "WARREN": 0, "REVELLE": 0, "SIXTH": 0, "UPPERS": 0,
          "LOWERS": 0, "VILLAGE": 0, "SIXTY FOUR": 0, "PC": 0, "GEISEL": 0, "BIOMED": 0}

for value in wifi_ucsd['Where have you experienced the worst quality WiFi on campus this quarter?']:
    split = value.split(", ")
    for key in places:
        for location in split:
            if location == key:
                places[key] += 1
places['BIOMED'] = 2
places

In [ ]:
columns = {"Location": list(places.keys()), "Count": list(places.values())}
locations = pd.DataFrame(columns)
locations.to_csv('location_data.csv', header=False, index=False)

In [ ]:
wifi_ucsd.head()

In [ ]:
re.search(r'[Bb]iomed', 'Galbraith, PC, Biomedical Library, Bottom Floor Geisel') != None

In [ ]:
places.values()

In [ ]:
total